In [1]:
import glob
import ast
import subprocess
import datetime
from pprint import pprint

In [2]:

project_name_vanila =  "exp_vanila"
project_name_personality_on = "exp_{agent_kind}_{personality}"


personality_sentences = {
    "OH" : "You are imaginative, creative, artistically appreciative, aesthetic, reflective, emotionally aware, curious, spontaneous, intelligent, analytical, sophisticated, and socially progressive. ",
    "OL" : "You are unimaginative, uncreative, artistically unappreciative, unaesthetic, unreflective, emotionally closed, uninquisitive, predictable, unitteligent, unanalytical, unsophisticated, and socially conservative. ",
    "CH" : "You are self-efficacious, orderly, responsible, hardworking, self-disciplined, practical, thrifty, organized, conscientious, and thorough. ",
    "CL" : "You are unsure, messy, irresponsible, lazy, undisciplined, impractical, extravagant, disorganized, negligent, and careless. ",
    "EH" : "You are friendly, extraverted, talkative, bold, assertive, active, energetic, adventurous, daring, and cheerful. ",
    "EL" : "You are unfriendly, introverted, silent, timid, unassertive, inactive, unenergetic, unadventurous, and gloomy. ",
    "AH" : "You are trustful, moral, honest, kind, generous altruistic, cooperative, humble, sympathetic, unselfish, and agreeable. ",
    "AL" : "You are distrustful, immoral, dishonest, unkind, stingy, unaltruistic, uncooperative, self-important, unsympathetic, selfish, and disagreeable. ",
    "NH" : "You are tense, nervous, anxious, angry, irritable, depressed, self-conscious, impulsive, discontented, and emotionally unstable. ",
    "NL" : "You are relaxed, at ease, easygoing, calm, patient, happy, unselfconscious, level-headed, contented, and emotionally stable. ",
}
agents_kinds = {
    "product_manager" : "D",
    "architect" : "A",
    "project_manager" : "J",
    "engineer" : "E",
}

In [3]:
project_name = "try_20231225_1"
#project_name = "try_20240111_1"
project_name = "try_20231224_1"
project_name = "try_20240117_0"
project_name = "preexp_240516_vanila_E_NL"
project_name = project_name_vanila.format(expiter=0)
#project_name = "otameshi"
N = 46
Nl = 33
Nr = 34
#N = 164

In [4]:
entry_points = []
with open(f"dataset_exp/humaneval/entry_point.txt", 'r') as f:
    for _ in range(164):
        entry_points.append(f.readline().strip())
print(entry_points)

['has_close_elements', 'separate_paren_groups', 'truncate_number', 'below_zero', 'mean_absolute_deviation', 'intersperse', 'parse_nested_parens', 'filter_by_substring', 'sum_product', 'rolling_max', 'make_palindrome', 'string_xor', 'longest', 'greatest_common_divisor', 'all_prefixes', 'string_sequence', 'count_distinct_characters', 'parse_music', 'how_many_times', 'sort_numbers', 'find_closest_elements', 'rescale_to_unit', 'filter_integers', 'strlen', 'largest_divisor', 'factorize', 'remove_duplicates', 'flip_case', 'concatenate', 'filter_by_prefix', 'get_positive', 'is_prime', 'find_zero', 'sort_third', 'unique', 'max_element', 'fizz_buzz', 'sort_even', 'decode_cyclic', 'prime_fib', 'triples_sum_to_zero', 'car_race_collision', 'incr_list', 'pairs_sum_to_zero', 'change_base', 'triangle_area', 'fib4', 'median', 'is_palindrome', 'modp', 'decode_shift', 'remove_vowels', 'below_threshold', 'add', 'same_chars', 'fib', 'correct_bracketing', 'monotonic', 'common', 'largest_prime_factor', 'sum

In [5]:
test_template = \
"""
import sys
sys.path.append('.')

from results.{proj_name}.{problem_name_and_time}.{code_dir_name}.{code_file_name} import {function_name}
from dataset_exp.humaneval.test.humaneval_{problem_index} import check

check({function_name})
print('PASS')
"""

In [ ]:
# 特定の番号だけ
project_name = project_name_vanila
#project_name = project_name_vanila.format(expiter=0)

# project_name = project_name_personality_on.format(
#     agent_kind=agents_kinds["project_manager"],  # engineer , product_manager,  architect,  project_manager
#     personality="NL", 
#     )


results = []
i = 0
code_file_name_ = "main"
#for i in range(N):
for _ in range(1):
    try:
        code_path = [n for n in glob.glob(f'./results/{project_name}/pr{i}_*/*/*.py')][0]
        print(code_path)
        print(code_path.split('/')[-1].split('.')[0])

        pprint(code_path)
        print([f.split('.')[1].split('/')[2] for f in [f for f in glob.glob(f'./results/{project_name}/pr{i}_*/*/*.py')]])
        for x in [f.split('.')[1].split('/')[3:] for f in [f for f in glob.glob(f'./results/{project_name}/pr{i}_*/*/*.py')]]:
            print(x)


        code_file_name = code_path.split('/')[-1].split('.')[0]
    except:
        results.append("fail")
        continue
    path_split = code_path.split('/')
    path_split.reverse()
    problem_name_and_time = path_split[2]
    code_dir_name = path_split[1]
    code_file_name = code_file_name_

    code = test_template.format(
        proj_name = project_name,
        problem_name_and_time = problem_name_and_time,
        code_dir_name = code_dir_name,
        code_file_name = code_file_name, #"main",
        function_name = entry_points[i],
        problem_index = i
    )
    print(f"problem_{i}=============================================")
    print(code)
    print(f"--------------------------------------------------------")
    if False:
        compiled = compile(code, '', 'exec')
        try:
            exec(compiled)
            results.append("accept")
        except:
            results.append("fail")
    if True:
        with open("tmp_test.py", "w") as f:
            f.write(code)
        try:
            result = subprocess.run(args=["python3.11",
                                            "tmp_test.py",
                                            ], capture_output=True, text=True, timeout=10)
            
            print(result.stdout)
            print(result.stderr)
        except Exception as e:
            print(f"ERROR:{e}")

        print(len(result.stdout)+len(result.stderr))
print(results)

In [ ]:
# [Nl, Nr) 番の問題をテスト

results = []
#for i in range(N):
for i in range(Nl, Nr, 1):
    try:
        code_path = [n for n in glob.glob(f'./results/{project_name}/pr{i}_*/*/main.py')][0]
    except:
        results.append("fail")
        continue
    path_split = code_path.split('/')
    path_split.reverse()
    problem_name_and_time = path_split[2]
    code_dir_name = path_split[1]

    code = test_template.format(
        proj_name = project_name,
        problem_name_and_time = problem_name_and_time,
        code_dir_name = code_dir_name,
        code_file_name = "main",
        function_name = entry_points[i],
        problem_index = i
    )
    print(f"problem_{i}=============================================")
    print(code)
    print(f"--------------------------------------------------------")
    if False:
        compiled = compile(code, '', 'exec')
        try:
            exec(compiled)
            results.append("accept")
        except:
            results.append("fail")
    if True:
        with open("tmp_test.py", "w") as f:
            f.write(code)
        try:
            result = subprocess.run(args=["python3.11",
                                            "tmp_test.py",
                                            ], capture_output=True, text=True, timeout=10)
            
            print(result.stdout)
            print(result.stderr)
        except Exception as e:
            print(f"ERROR:{e}")

        print(len(result.stdout)+len(result.stderr))
print(results)